#### Day 15 - B
Complete the robot instructions and monitor the warehouse layout.

The grid is now doubled in size.

In [113]:
#Import Libraries and settings
from copy import deepcopy

settings = {
    "day": 15,
    "test_data": 0
}

In [114]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    #Save grid to 2D array and inputs to seperate array
    at_grid = True
    inputs = []
    grid = []
    for idx_y, line in enumerate(lines):

        #Check if the grid has finished
        if line == "":
            at_grid = False
        else:
            if at_grid:
                line_large = ""
                for char in line:
                    if char == "#":
                        line_large += "##"
                    elif char == ".":
                        line_large += ".."
                    elif char == "O":
                        line_large += "[]"
                    else:
                        robot_loc = (len(line_large), idx_y)
                        line_large += "@."

                grid.append(list(line_large))
            else:
                inputs += list(line)

    return grid, inputs, robot_loc

grid_base, inputs, starting_loc = load_input(settings)

In [115]:
grid = deepcopy(grid_base)

In [116]:
def check_loc(grid, loc):
    return grid[loc[1]][loc[0]]

In [117]:
#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Apply a direction to a location
def apply_move(loc, dir, mag=1):
    step = translate_step(dir, mag)
    return (loc[0]+step[0], loc[1]+step[1])

#Test the next move
def try_next_move(grid, c_loc, dir):
    #Get character for current location
    c_char = check_loc(grid, c_loc)
    #Get the next location and character currently in the space
    next_loc = apply_move(c_loc, dir)
    next_char = check_loc(grid, next_loc)

    #If wall, do not return a move
    if (next_char == "#"):
        return []
    #If open space, return a move
    elif (next_char == "."):
        return [(c_char, c_loc, next_loc)]
    #If a box, check if the box can move and domino effect
    elif (next_char == "[") or (next_char == "]"):
        #If dir is < or >, handle as normal
        if dir == "<" or dir == ">":
            #Get the fallout from moving this box
            res = try_next_move(grid, next_loc, dir)
            #If no res then box cannot move
            if res == []:
                return []
            #If res then move robot along with box(es)
            else:
                return [(c_char, c_loc, next_loc)] + res
        #If dir is ^ or v then move both sections seperate and combine the result
        else:
            if next_char == "[":
                next_loc_l = next_loc
                next_loc_r = apply_move(next_loc, ">")
            else:
                next_loc_l = apply_move(next_loc, "<")
                next_loc_r = next_loc

            #Attempt to move both halves of the box seperately
            res_l = try_next_move(grid, next_loc_l, dir)
            res_r = try_next_move(grid, next_loc_r, dir)

            #If both can move perform moves from both
            if res_l and res_r:
                #Remove overlap from res
                res_l = [x for x in res_l if x not in res_r]

                return [(c_char, c_loc, next_loc)] + res_l + res_r
            else:
                return []


#Apply a list of moves to the grid
def apply_next_moves(grid, moves):
    
    #Apply moves in reverse
    for move in moves[::-1]:
        old_loc = move[1]
        new_loc = move[2]

        #Check if the character moving is the robot
        if move[0] == "@":
            #Set n_loc so the robot's position can be updated
            n_loc = new_loc

        grid[new_loc[1]][new_loc[0]] = grid[old_loc[1]][old_loc[0]]
        grid[old_loc[1]][old_loc[0]] = "."

    return n_loc

In [118]:
#Apply the input to the robot
def apply_instructions(grid, inputs, c_loc):
    for dir in inputs:
        moves = try_next_move(grid, c_loc, dir)
        if moves:
            c_loc = apply_next_moves(grid, moves)

def calculate_score(grid):
    subtotal = 0
    for idx_y, line in enumerate(grid):
        for idx_x, space in enumerate(line):
            if space == "[":
                subtotal += (100*idx_y) + idx_x

    return subtotal

In [119]:
apply_instructions(grid, inputs, starting_loc)

In [120]:
calculate_score(grid)

1538862

In [121]:
for line in grid:
    print("".join(line))

####################################################################################################
##[][]....[]..........[]..[][][]....[]..........[]........[].....[][]......[][][].......[][]..[]####
##..[][]........[]......[]......##..[]......[]..............##..##[]....[][][]..............[]######
##[]..[]..................[][]..[][][]..[]......##......##[]..##....[]..##.[].....##[]........[][]##
##..[]........[][]....[]..................................##[].[]........[].........[]##......[][]##
##[]....##..[]......[]......[]........[][]....[]........................[]......##[]##............##
##..##..[][]..##........##....[]##..........##....[]........[][]........[]..[]..##..[]............##
##......[]..[]..[]......[]......[]..[][][]......[]..........[][]..........................##....####
##[]..##[]....[][]......[][][]..[]..##[]......[][][]..[][]..........[]..........................[]##
##......[]..............[][][]....[]......######..[][]..##[]..[]............[].............